In [ ]:
# from pathlib import Path
# import os as os
# import pandas as pd
# from extract import extract_data, extract_test_data
# from utils import *
# from mapping import ReportMapper
# from initial_process import validate_rt30, validate_rmd
# from form_90_post_mapping import RT30PostMapper

# CURRENT_REPORTING_PERIOD = pd.to_datetime('2024-12-31')  

# def load_true_FI_AUX_OTH(load_file: Path):
#     schema_dict = {
#         'customer_nr': str,
#         'customer_name': str,
#         'Counterparty': str
#     }

#     with pd.ExcelFile(load_file) as xls:
#         true_FI_AUX_OTH_df = pd.read_excel(
#             xls, dtype=schema_dict)[[
#             'customer_nr', 'customer_name', 'Counterparty Type']]

#     return true_FI_AUX_OTH_df

# def update_rv_cpty_type(row, true_oth_df):
#     if row['customer_nr'] in true_oth_df['customer_nr'].values:
#         if row['rv_cpty_type_thisQ'] == 'FI-AUX-OTH':
#             row['rv_cpty_type_thisQ'] = 'FI-AUX-OTH'
#         elif row['rv_cpty_type_lastQ'] == 'FI-AUX-OTH':
#             row['rv_cpty_type_lastQ'] = 'FI-AUX-OTH'
#     else:
#         if row['rv_cpty_type_thisQ'] == 'FI-AUX-OTH':
#             row['rv_cpty_type_thisQ'] = 'FI-MM-INV-FUND'
#         elif row['rv_cpty_type_lastQ'] == 'FI-AUX-OTH':
#             row['rv_cpty_type_lastQ'] = 'FI-MM-INV-FUND'
#     return row

# def update_and_filter_rt30(main_df: pd.DataFrame, true_oth_df: pd.DataFrame):
#     # Check if the DataFrames are properly loaded
#     if main_df is None:
#         raise ValueError("rt30_df is None")
#     if true_oth_df is None:
#         raise ValueError("true_FI_AUX_OTH_df is None")

#     # Convert customer_nr in true_FI_AUX_OTH_df to string
#     true_oth_df['customer_nr'] = true_oth_df['customer_nr'].astype(str)

#     # Update rv_cpty_type_thisQ based on the conditions
#     main_df = main_df.apply(update_rv_cpty_type, axis=1, true_oth_df=true_oth_df)

#     # Filter and display the rows where customer_nr is in true_FI_AUX_OTH's customer_nr
#     filtered_df_rt30 = main_df[main_df['customer_nr'].isin(true_oth_df['customer_nr'])]

#     return main_df, filtered_df_rt30


In [2]:
# from pathlib import Path
# import os as os
# import pandas as pd
# from extract import extract_data, extract_test_data
# from utils import *
# from mapping import ReportMapper
# from initial_process import validate_rt30, validate_rmd
# from form_90_post_mapping import RT30PostMapper


# CURRENT_REPORTING_PERIOD = pd.to_datetime('2024-12-31')

# def load_true_FI_AUX_OTH(load_file: Path) -> pd.DataFrame:
#     """Load FI-AUX-OTH data with optimized reading"""
#     schema_dict = {
#         'customer_nr': str,
#         'customer_name': str,
#         'Counterparty': str
#     }
    
#     usecols = ['customer_nr', 'customer_name', 'Counterparty Type']
    
#     with pd.ExcelFile(load_file) as xls:
#         true_FI_AUX_OTH_df = pd.read_excel(
#             xls,
#             dtype=schema_dict,
#             usecols=usecols
#         )
    
#     return true_FI_AUX_OTH_df

# def update_and_filter_rt30(main_df: pd.DataFrame, true_oth_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
#     """Update and filter RT30 data using vectorized operations"""
    
#     # Input validation
#     if main_df is None or true_oth_df is None:
#         raise ValueError("Input DataFrames cannot be None")
    
#     # Convert customer_nr to string in both DataFrames
#     main_df['customer_nr'] = main_df['customer_nr'].astype(str)
#     true_oth_df['customer_nr'] = true_oth_df['customer_nr'].astype(str)
    
#     # Create mask for customers in true_oth_df
#     customer_mask = main_df['customer_nr'].isin(true_oth_df['customer_nr'])
    
#     # Update thisQ values
#     thisQ_mask = main_df['rv_cpty_type_thisQ'] == 'FI-AUX-OTH'
#     main_df.loc[thisQ_mask & ~customer_mask, 'rv_cpty_type_thisQ'] = 'FI-MM-INV-FUND'
    

#     # Update lastQ values
#     lastQ_mask = main_df['rv_cpty_type_lastQ'] == 'FI-AUX-OTH'
#     main_df.loc[lastQ_mask & ~customer_mask, 'rv_cpty_type_lastQ'] = 'FI-MM-INV-FUND'
    
#     # Filter rows
#     filtered_df_rt30 = main_df[customer_mask]
    
#     return main_df, filtered_df_rt30

In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os as os
import pandas as pd
from extract import extract_data, extract_test_data
from utils import ReportingLogger, log_errors
from mapping import ReportMapper
import numpy as np
from form_90_post_mapping import *

current_rpt_period = pd.to_datetime('2024-12-31')

def load_true_aux_oth(load_file):
    schema_dict = {
        'customer_nr': str,
        'customer_name':str,
        'Counterparty': str
    }

    with pd.ExcelFile(load_file) as xls:
        true_aux_oth_df = pd.read_excel(xls, 'Counterparty type FI-AUX-OTH', dtype=schema_dict)

    return true_aux_oth_df

   
def update_and_filter_rt30(main_df, true_aux_oth_df) -> pd.DataFrame:

    if main_df is None or true_aux_oth_df is None:
        return ValueError('Inpur Dataframes are empty')
    
    main_df['customer_nr'] = main_df['customer_nr'].astype(str)
    true_aux_oth_df['customer_nr'] = true_aux_oth_df['customer_nr'].astype(str)

    customer_mask = main_df['customer_nr'].isin(true_aux_oth_df['customer_nr'])

    customer_mask = main_df['customer_nr'].isin(true_aux_oth_df['customer_nr'])

    thisQ_mask = main_df['rv_cpty_type_thisQ'] == 'FI-AUX-OTH'
    
    lastQ_mask = main_df['rv_cpty_type_lastQ'] == 'FI-AUX-OTH'

    main_df['rv_cpty_type_thisQ'] = np.where(
    (thisQ_mask & ~customer_mask),
    'FI-MM-INV-FUND',
    main_df['rv_cpty_type_thisQ']
    )

    main_df['rv_cpty_type_lastQ'] = np.where(
    (lastQ_mask & ~customer_mask),
    'FI-MM-INV-FUND',
    main_df['rv_cpty_type_lastQ']
    )

    return main_df

        
curr_path = Path(os.getcwd())
    
output_path = curr_path.parent/ 'output'

data_path = curr_path.parent/'data'

load_file = data_path / 'Counterparty_type_FI_AUX_OTH.xlsx'

aux_oth=load_true_aux_oth(load_file)

rt30_raw = extract_test_data()

rt30_df = rt30_raw.get('rt30')

updated_rt30_df = update_and_filter_rt30(rt30_df, aux_oth)

    
    # updated_df_rt30['rv_mat_remaining'] = updated_df_rt30['rv_mat_remaining'].astype(str).str.rstrip('0').str.rstrip('.')

rt30_mapper = ReportMapper(curr_path.parent/'config'/ 'RT30.json')

df_rt30_mapped = rt30_mapper.map_data(updated_rt30_df)

post_mapper = RT30PostMapper(df_rt30_mapped,current_rpt_period)
processed_data = post_mapper.process_all()

# with pd.ExcelWriter(output_path / 'raw_data.xlsx') as writer:
#             processed_data.to_excel(writer, sheet_name='mapped_rt30', index=False)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-03-08 17:26:42,209 - mapping - INFO - Initializing ReportMapper with config: c:\Users\61430\project_workspace\BOC-reporting\90\config\RT30.json
2025-03-08 17:26:42,210 - mapping - INFO - Configuration loaded successfully
2025-03-08 17:26:42,211 - mapping - INFO - Loading mappings from: c:\Users\61430\project_workspace\BOC-reporting\90\data\all_mappings.xlsx
2025-03-08 17:26:42,270 - mapping - INFO - Mappings loaded successfully
2025-03-08 17:26:42,271 - mapping - INFO - Starting mapping process
2025-03-08 17:26:42,283 - mapping - INFO - Applying mapping 1/6: Intercompany type- FORM90
2025-03-08 17:26:42,284 - mapping - INFO - Applying mapping using keys: rv_rel_party_type_lastQ -> Intercompany
2025-03-08 17:26:42,312 - mapping - WARNING - Sheet 'Intercompany type- FORM90': 1472 NaN values found in mapping key '['Intercompany']'. The mapping sheet may be out of date.
2025-03-08 17:26:42,323 - mapping - INFO - Applying mapping 2/6: Intercompany type- FORM90
2025-03-08 17:26:42,323 -

In [4]:
processed_data[processed_data['ide_linkage_ref'] == '9512300019071101']['Status']


1747    D
Name: Status, dtype: object

In [1]:
from pathlib import Path
import os as os
import pandas as pd
from extract import extract_data, extract_test_data
from utils import ReportingLogger, log_errors
from mapping import ReportMapper
import numpy as np
from form_90_post_mapping import *

curr_path = Path(os.getcwd())

path = curr_path.parent/'data'/'Form90 BOCS Working NEW 20241231 updated.xlsb'


na_values = ['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A N/A', '#N/A', 'N/A',
                'n/a', '<NA>', '#NA', 'null', 'NaN', '-NaN', 'nan', '-nan', 'None', '']

schema_dict = {
    'lot_type_fk': str,
    'ide_linkage_ref': str,
    'ide_linkage_type': str,
    'ide_sourcesys_ref': str,
    'customer_nr': str,
    'ISIN': str,
    'rv_mat_remaining': str
}


output = {}

output['rt30'] = pd.read_excel(
    path,
    sheet_name='RT30',
    dtype=schema_dict,
    keep_default_na=False,
    # parse_dates=['value_date', 'maturity_date'],
    na_values=na_values
)[['lot_type_fk',
    'ide_linkage_ref',
    'ide_linkage_type',
    'ide_sourcesys_ref',
    'dealtype',
    'customer_nr',
    'customer_name',
    'rv_coa',
    'rv_cpty_type_lastQ',
    'rv_cpty_type_thisQ',
    'rv_rel_party_type_lastQ',
    'rv_rel_party_type_thisQ',
    'rv_resident_lastQ',	
    'rv_resident_thisQ',	
    'nationality_lastQ',	
    'nationality_thisQ',	
    'domicile_lastQ',	
    'domicile_thisQ',	
    'currency',	
    'value_date',	
    'maturity_date',	
    'fx_rate_value',	
    'fx_rate_maturity',	
    'rv_mat_original',	
    'rv_mat_remaining',	
    'nominal',	
    'trade_price',	
    'ISIN',	
    'rca_bookv_thisQ',	
    'rca_ori_bookv_thisQ',	
    'rca_marketv_thisQ',	
    'rca_ori_marketv_thisQ',	
    'rca_accrint_thisQ',	
    'rca_prov_coll_thisQ',
    'rca_prov_indi_thisQ',	
    'fx_rate_thisQ',	
    'rca_bookv_lastQ',	
    'rca_ori_bookv_lastQ',	
    'rca_marketv_lastQ',	
    'rca_ori_marketv_lastQ',	
    'rca_accrint_lastQ',	
    'rca_prov_coll_lastQ',	
    'rca_prov_indi_lastQ',	
    'fx_rate_lastQ']]



output['rt30'].columns = output['rt30']\
    .columns.str.split('.').str[0]

output['rt30']['maturity_date_status'] = output['rt30']['maturity_date']
output['rt30']['value_date_status'] = output['rt30']['value_date']

output['rt30']['maturity_date_status'].replace({"NULL": np.nan, 2958465: 132320}, inplace=True)
output['rt30']['value_date_status'].replace({"NULL": np.nan, 2958465: 132320}, inplace=True)

output['rt30']['maturity_date_status'] = pd.to_datetime(output['rt30']['maturity_date_status'],origin=pd.Timestamp('1899-12-30'), unit='D')
output['rt30']['value_date_status'] = pd.to_datetime(output['rt30']['value_date_status'],origin=pd.Timestamp('1899-12-30'), unit='D')



In [8]:
output['rt30']['maturity_date_status'] = pd.to_datetime(output['rt30']['maturity_date_status'].astype(float),origin=pd.Timestamp(1899, 12, 30), unit='D')




TypeError: float() argument must be a string or a real number, not 'NaTType'

In [2]:
output['rt30']['maturity_date_status']

0       2262-04-11
1       2262-04-11
2       2262-04-11
3       2262-04-11
4       2262-04-11
           ...    
20246   2024-12-31
20247   2024-12-31
20248   2024-12-31
20249   2024-12-31
20250   2024-12-31
Name: maturity_date_status, Length: 20251, dtype: datetime64[ns]

In [ ]:
def analyze_status(data: pd.DataFrame, ide_linkage_ref: str) -> None:
    """Analyze and display status logic for a specific record"""
    
    # Get the specific record
    mask = data['ide_linkage_ref'] == ide_linkage_ref
    if not data[mask].empty:
        record = data[mask].iloc[0]
        
        # Format dates
        maturity_date = pd.to_datetime(record['maturity_date'])
        current_period = pd.to_datetime('2024-12-31')
        
        # Print analysis header
        print("Status Logic Analysis")
        print("="*50)
        
        # 1. Display Input Values
        print("\n1. Input Values:")
        print(f"{'Maturity Date:':<20} {maturity_date:%Y-%m-%d}")
        print(f"{'Current Period:':<20} {current_period:%Y-%m-%d}")
        print(f"{'Book Value ThisQ:':<20} {record['rca_bookv_thisQ']:,.2f}")
        print(f"{'Book Value LastQ:':<20} {record['rca_bookv_lastQ']:,.2f}")
        
        # 2. Status Conditions Check
        print("\n2. Status Conditions:")
        
        # Check Matured (M) conditions
        m_date_check = maturity_date <= current_period
        m_book_check = record['rca_bookv_thisQ'] == 0
        is_matured = m_date_check and m_book_check
        
        print("\nMatured (M) Check:")
        print(f"✓ Date <= Current:    {m_date_check}")
        print(f"✓ BookV ThisQ = 0:    {m_book_check}")
        print(f"→ Matured Status:     {is_matured}")
        
        # Check Default (D) conditions
        d_date_check = maturity_date > current_period
        d_book_check = record['rca_bookv_thisQ'] == 0
        is_default = d_date_check and d_book_check
        
        print("\nDefault (D) Check:")
        print(f"✓ Date > Current:     {d_date_check}")
        print(f"✓ BookV ThisQ = 0:    {d_book_check}")
        print(f"→ Default Status:     {is_default}")
        
        # Check New (N) condition
        is_new = record['rca_bookv_lastQ'] == 0
        print("\nNew (N) Check:")
        print(f"✓ BookV LastQ = 0:    {is_new}")
        
        # 3. Final Status Determination
        print("\n3. Status Determination:")
        if is_matured:
            expected_status = 'M'
            reason = "Matured: Date ≤ Current Period AND BookV ThisQ = 0"
        elif is_default:
            expected_status = 'D'
            reason = "Default: Date > Current Period AND BookV ThisQ = 0"
        elif is_new:
            expected_status = 'N'
            reason = "New: BookV LastQ = 0"
        else:
            expected_status = 'O'
            reason = "Other: No specific conditions met"
            
        print(f"Expected Status: {expected_status}")
        print(f"Reason: {reason}")
        print(f"Actual Status:  {record['Status']}")
        
        if expected_status != record['Status']:
            print("\n⚠️ Warning: Expected status differs from actual status!")
    else:
        print(f"No record found with ide_linkage_ref: {ide_linkage_ref}")

# Execute the analysis
analyze_status(processed_data, '9512300019071101')

Status Logic Analysis

1. Input Values:
Maturity Date:       1970-01-01
Current Period:      2024-12-31
Book Value ThisQ:    0.00
Book Value LastQ:    165,024,586.45

2. Status Conditions:

Matured (M) Check:
✓ Date <= Current:    True
✓ BookV ThisQ = 0:    True
→ Matured Status:     True

Default (D) Check:
✓ Date > Current:     False
✓ BookV ThisQ = 0:    True
→ Default Status:     False

New (N) Check:
✓ BookV LastQ = 0:    False

3. Status Determination:
Expected Status: M
Reason: Matured: Date ≤ Current Period AND BookV ThisQ = 0
Actual Status:  M


In [ ]:

class RT30PostMapper:
    """Handles post-mapping logic for RT30 and RT32 reports"""

    def __init__(self, data, mapper=None):
        self.data = data.copy()
        self.mapper = mapper

    def Domicile_thisQ(self, row):
        if row['rv_resident_thisQ'] == 'NO' and row['nationality_thisQ'] == 'AU':
            return 'CN'
        elif row['rv_resident_thisQ'] == 'NO' and (row['domicile_thisQ'] == 'YY' or row['domicile_thisQ'] == 'XX'):
            return row['nationality_thisQ']
        else:
            return row['domicile_thisQ']

    def apply_Domicile_thisQ(self):
        self.data['Domicile_thisQ'] = self.data.apply(self.Domicile_thisQ, axis=1)
        return self

    def Domicile_lastQ(self, row):
        if row['rv_resident_lastQ'] == 'NO' and row['nationality_lastQ'] == 'AU':
            return 'CN'
        elif row['rv_resident_lastQ'] == 'NO' and (row['domicile_lastQ'] == 'YY' or row['domicile_lastQ'] == 'XX'):
            return row['rv_resident_lastQ']
        else:
            return row['domicile_lastQ']

    def apply_Domicile_lastQ(self):
        self.data['Domicile_lastQ'] = self.data.apply(self.Domicile_lastQ, axis=1)
        return self

    def Intercompany_type_thisQ(self, row):
        if row['rv_resident_thisQ'] == 'NULL' or row['Domicile_thisQ'] == 'NULL':
            return ''
        elif row['rv_resident_thisQ'] == 'YES' or row['Domicile_thisQ'] == 'XX':
            return 'NA'
        else:
            return row['Intercompany_type_thisQ_inter_var']

    def apply_Intercompany_type_thisQ(self):
        self.data['Intercompany_type_thisQ'] = self.data.apply(self.Intercompany_type_thisQ, axis=1)
        return self

    def Intercompany_type_lastQ(self, row):
        if row['rv_resident_lastQ'] == 'NULL' or row['Domicile_lastQ'] == 'NULL':
            return ''
        elif row['rv_resident_lastQ'] == 'YES' or row['Domicile_lastQ'] == 'XX':
            return 'NA'
        else:
            return row['Intercompany_type_lastQ_inter_var']

    def apply_Intercompany_type_lastQ(self):
        self.data['Intercompany_type_lastQ'] = self.data.apply(self.Intercompany_type_lastQ, axis=1)
        return self

    def Counterparty_type_thisQ(self, row):
        if row['rv_resident_thisQ'] == 'NULL' or row['Domicile_thisQ'] == 'NULL':
            return ''
        elif row['rv_resident_thisQ'] == 'YES' or row['Domicile_thisQ'] == 'XX':
            return 'NA'
        else:
            return row['Intercompany_type_thisQ_inter_var']

    def apply_Counterparty_type_thisQ(self):
        self.data['Counterparty_type_thisQ'] = self.data.apply(self.Counterparty_type_thisQ, axis=1)
        return self

    def Counterparty_type_lastQ(self, row):
        if row['rv_resident_lastQ'] == 'NULL' or row['Domicile_lastQ'] == 'NULL':
            return ''
        elif row['rv_resident_lastQ'] == 'YES' or row['Domicile_lastQ'] == 'XX':
            return 'NA'
        else:
            return row['Intercompany_type_lastQ_inter_var']

    def apply_Counterparty_type_lastQ(self):
        self.data['Counterparty_type_lastQ'] = self.data.apply(self.Counterparty_type_lastQ, axis=1)
        return self

    def Customer_diff(self, row):
        if (row['Counterparty_type_lastQ'] == '' or row['Counterparty_type_thisQ'] == '' or
            row['Intercompany_type_lastQ'] == '' or row['Intercompany_type_thisQ'] == '' or
            (row['Counterparty_type_lastQ'] == row['Counterparty_type_thisQ'] and
             row['Intercompany_type_lastQ'] == row['Intercompany_type_thisQ'] and
             row['Domicile_lastQ'] == row['Domicile_thisQ'])):
            return 'NO'
        else:
            return 'YES'
            
    def Counterparty_type_thisQ(self, row):
        if row['rv_resident_thisQ'] == 'NULL' or row['Domicile_thisQ'] == 'NULL':
            return ''
        elif row['rv_resident_thisQ'] == 'YES' or row['Domicile_thisQ'] == 'XX':
            return 'NA'
        else:
            return row['Intercompany_type_thisQ_inter_var']

    def apply_Counterparty_type_thisQ(self):
        self.data['Counterparty_type_thisQ'] = self.data.apply(self.Counterparty_type_thisQ, axis=1)
        return self

    def Counterparty_type_lastQ(self, row):
        if row['rv_resident_lastQ'] == 'NULL' or row['Domicile_lastQ'] == 'NULL':
            return ''
        elif row['rv_resident_lastQ'] == 'YES' or row['Domicile_lastQ'] == 'XX':
            return 'NA'
        else:
            return row['Intercompany_type_lastQ_inter_var']

    def apply_Counterparty_type_lastQ(self):
        self.data['Counterparty_type_lastQ'] = self.data.apply(self.Counterparty_type_lastQ, axis=1)
        return self

    def apply_Customer_diff(self):
        self.data['Customer_diff'] = self.data.apply(self.Customer_diff, axis=1)
        return self
    
    def Original_maturity(self, row):
        """
        Determines original maturity based on maturity date and rv_mat_original
        Returns 'Short-term' or 'Long-term' based on conditions
        """
        try:
            # Convert maturity_date to string for comparison
            maturity_date_str = str(row['maturity_date']).strip()
            
            # Check conditions without datetime conversion
            if (maturity_date_str in ['12/31/9999', '9999-12-31', 'NULL', 'NaT', ''] or 
                str(row['rv_mat_original']) == 'NULL' or
                pd.isna(row['maturity_date']) or
                float(str(row['rv_mat_original']).replace('NULL', '0')) <= 1012000):
                return 'Short-term'
            else:
                return 'Long-term'
        except:
            return 'Short-term'  # Default to Short-term on any error

    def apply_Original_maturity(self):
        """Applies Original_maturity calculation to DataFrame"""
        self.data['Original_maturity'] = self.data.apply(self.Original_maturity, axis=1)
        return self
    
    def Residual_maturity(self, row):
        """
        Determines residual maturity based on maturity date
        Returns 'Term<= 90 days' or the value from Residual maturity_inter_var
        """
        try:
            # Check for specific date string without converting to datetime
            if str(row['maturity_date']) in ['12/31/9999', 'NULL', 'NaT', '']:
                return 'Term<= 90 days'
            
            # Try to convert to datetime for other cases
            maturity_date = pd.to_datetime(row['maturity_date'])
            if pd.isna(maturity_date):
                return 'Term<= 90 days'
            else:
                return row['Residual maturity_inter_var']
        except:
            return 'Term<= 90 days'

    def apply_Residual_maturity(self):
        """Applies Residual_maturity calculation to DataFrame"""
        self.data['Residual_maturity'] = self.data.apply(self.Residual_maturity, axis=1)
        return self
    
    def convert_nominal(self, row):
        """
        Converts nominal values based on NULL condition
        Returns 0 if nominal is NULL, otherwise returns nominal value
        """
        try:
            if str(row['nominal']).upper() == 'NULL' or pd.isna(row['nominal']):
                return 0
            return float(row['nominal'])
        except:
            return 0  # Default to 0 on any error

    def apply_nominal_conversion(self):
        """Applies nominal conversion to DataFrame"""
        self.data['nominal_converted'] = self.data.apply(self.convert_nominal, axis=1)
        return self
    
    def get_data(self):
        return self.data
    
    def process_all(self):
          
    # Chains all transformations in sequence and returns the processed DataFrame
  
        return (self
                .apply_Domicile_thisQ()
                .apply_Domicile_lastQ()
                .apply_Intercompany_type_thisQ()
                .apply_Intercompany_type_lastQ()
                .apply_Counterparty_type_thisQ()
                .apply_Counterparty_type_lastQ()
                .apply_Customer_diff()
                .apply_Original_maturity()
                .apply_Residual_maturity() 
                .apply_nominal_conversion()
                .get_data())

# Example usage
# Assuming df is your DataFrame
# df = pd.DataFrame({...})

# Create an instance of the class
post_mapper = RT30PostMapper(df_rt30_mapped)
processed_data = post_mapper.process_all()



# Display the processed data
print(processed_data)



      lot_type_fk                 ide_linkage_ref ide_linkage_type  \
0            1008                 100000500001730                2   
1            1008                 100000500001785                2   
2            1008                 100000500001865                2   
3            1008                 100000500014922                2   
4            1008                 100000500033127                2   
...           ...                             ...              ...   
20246        1008  6201-DR-51013-51230-CAD-0-BOCS                1   
20247        1008    5403-DR-51250-51264-CNY-0-GL                1   
20248        1008    5403-DR-51269-51269-NZD-0-GL                1   
20249        1008        5403-DR-51930-0-SGD-0-GL                1   
20250        1008    5403-DR-51930-51930-CNY-0-GL                1   

      ide_sourcesys_ref       dealtype customer_nr  \
0                   100  L01-同业存入_8304  6022004055   
1                   100  L01-同业存入_8023  6022004134 

      lot_type_fk                 ide_linkage_ref ide_linkage_type  \
0            1008                 100000500001730                2   
1            1008                 100000500001785                2   
2            1008                 100000500001865                2   
3            1008                 100000500014922                2   
4            1008                 100000500033127                2   
...           ...                             ...              ...   
20246        1008  6201-DR-51013-51230-CAD-0-BOCS                1   
20247        1008    5403-DR-51250-51264-CNY-0-GL                1   
20248        1008    5403-DR-51269-51269-NZD-0-GL                1   
20249        1008        5403-DR-51930-0-SGD-0-GL                1   
20250        1008    5403-DR-51930-51930-CNY-0-GL                1   

      ide_sourcesys_ref       dealtype customer_nr  \
0                   100  L01-同业存入_8304  6022004055   
1                   100  L01-同业存入_8023  6022004134 

     lot_type_fk  ide_linkage_ref ide_linkage_type ide_sourcesys_ref  \
18           992  100000500429290                2               100   
19           992  100000500468819                2               100   
77          1008  100000500027919                2               100   
91          1008  100000500156707                2               100   
116         1008  100000500025650                2               100   
...          ...              ...              ...               ...   
6855        1008  100000500491430                5               100   
6884        1008        330134556                5                22   
6903        1008        329573239                5                22   
7001        1008        332003015                5                22   
7051        1008        328064519                5                22   

           dealtype customer_nr                               customer_name  \
18    L01-同业存入_8303   559631763               CMB Wealth

18      FI-AUX-OTH
19      FI-AUX-OTH
77      FI-AUX-OTH
91      FI-AUX-OTH
116     FI-AUX-OTH
           ...    
6855    FI-AUX-OTH
6884    FI-AUX-OTH
6903    FI-AUX-OTH
7001    FI-AUX-OTH
7051    FI-AUX-OTH
Name: rv_cpty_type_thisQ, Length: 223, dtype: object


In [ ]:
print(len(rt30_df['customer_nr']))

20251


# Aggregation 731.1a and 731.1l


In [ ]:
from aggregation import ReportAggregator731AL

aggregator = ReportAggregator731AL(df_rt30_output)
df_731a = aggregator.generate_731a(df_algl)
df_731l = aggregator.generate_731l(df_algl)


2025-01-16 14:16:53,952 - aggregation - INFO - Initializing ReportAggregator
2025-01-16 14:16:53,953 - aggregation - INFO - Starting 731.1a assets report aggregation
2025-01-16 14:16:54,843 - aggregation - INFO - 731.1a assets report completed successfully
2025-01-16 14:16:54,844 - aggregation - INFO - Starting 731.1l liabilities report aggregation
2025-01-16 14:16:56,051 - aggregation - INFO - 731.1l liabilities report completed successfully


In [ ]:
yaml_file = '../config/RT30_RT32.yaml'
mapper = ReportMapper(yaml_file)

2025-01-16 16:10:04,251 - mapping - INFO - Initializing ReportMapper with config: ../config/RT30_RT32.yaml
2025-01-16 16:10:04,262 - mapping - INFO - Configuration loaded successfully
2025-01-16 16:10:04,263 - mapping - INFO - Using cached mappings for: d:\study\BoC_reporting\BOC-reporting\data\all_mappings.xlsx
2025-01-16 16:10:04,264 - mapping - INFO - Mappings loaded successfully


In [ ]:
from aggregation import ReportAggregator731B
mapper.reload_mappings()
aggregator = ReportAggregator731B(ReportMapper, df_rt30_output)
df_731b = aggregator\
    .pre_ob_process(df_algl, df_facility_output)


2025-01-16 19:42:26,981 - mapping - INFO - Reloading mappings from: d:\study\BoC_reporting\BOC-reporting\data\all_mappings.xlsx
2025-01-16 19:42:26,985 - mapping - INFO - Loading mappings from: d:\study\BoC_reporting\BOC-reporting\data\all_mappings.xlsx
2025-01-16 19:42:27,515 - mapping - INFO - Mappings loaded successfully
2025-01-16 19:42:27,517 - mapping - INFO - Mappings reloaded successfully


AttributeError: type object 'ReportMapper' has no attribute 'all_mappings'

In [ ]:
def add_top_header(df, header, start=2):
    multi_index = [('' if col not in df.columns[start:]
                    else header, col) for col in df.columns]
    df.columns = pd.MultiIndex.from_tuples(multi_index)

In [ ]:
result = df_731b.result
result['Part E']


for k, v in result.items():
    if k == "Part A":
        add_top_header(v, k, 3)
    else:
        add_top_header(v, k)

In [ ]:
df_731b.post_ob_process(df_ob_check)

Part A               \
    Vis-à-vis country Vis-à-vis country name GL adj <=3 mths 3m-12m 1y-2y   
0                  AF            Afghanistan    NaN      NaN    NaN   NaN   
1                  AL                Albania    NaN      NaN    NaN   NaN   
2                  DZ                Algeria    NaN      NaN    NaN   NaN   
3                  AS         American Samoa    NaN      NaN    NaN   NaN   
4                  AD                Andorra    NaN      NaN    NaN   NaN   
..                ...                    ...    ...      ...    ...   ...   
235                WF      Wallis and Futuna    NaN      NaN    NaN   NaN   
236                EH         Western Sahara    NaN      NaN    NaN   NaN   
237                YE                  Yemen    NaN      NaN    NaN   NaN   
238                ZM                 Zambia    NaN      NaN    NaN   NaN   
239                ZW               Zimbabwe    NaN      NaN    NaN   NaN   

                                                ...  \
    2y-5y 5y+ Maturity bucket Adjustment Total  ...   
0     NaN NaN                        NaN   NaN  ...   
1     NaN NaN                        NaN   NaN  ...   
2     NaN NaN                        NaN   NaN  ...   
3     NaN NaN                        NaN   NaN  ...   
4     NaN NaN                        NaN   NaN  ...   
..    ...  ..                        ...   ...  ...   
235   NaN NaN                        NaN   NaN  ...   
236   NaN NaN                        NaN   NaN  ...   
237   NaN NaN                        NaN   NaN  ...   
238   NaN NaN                        NaN   NaN  ...   
239   NaN NaN                        NaN   NaN  ...   

                        Part D                                         \
    Non-financial corporations Households and non-profits Unallocated   
0                          NaN                        NaN         NaN   
1                          NaN                        NaN         NaN   
2                          NaN                        NaN         NaN   
3                          NaN                        NaN         NaN   
4                          NaN                        NaN         NaN   
..                         ...                        ...         ...   
235                        NaN                        NaN         NaN   
236                        NaN                        NaN         NaN   
237                        NaN                        NaN         NaN   
238                        NaN                        NaN         NaN   
239                        NaN                        NaN         NaN   

                                                             \
    Total international claims Check Vis-à-vis country name   
0                          NaN   NaN            Afghanistan   
1                          NaN   NaN                Albania   
2                          NaN   NaN                Algeria   
3                          NaN   NaN         American Samoa   
4                          NaN   NaN                Andorra   
..                         ...   ...                    ...   
235                        NaN   NaN      Wallis and Futuna   
236                        NaN   NaN         Western Sahara   
237                        NaN   NaN                  Yemen   
238                        NaN   NaN                 Zambia   
239                        NaN   NaN               Zimbabwe   

                             Part E                                    
    Derivative contracts Guarantees Credit commitments Guarantees adj  
0                    NaN        NaN                NaN            NaN  
1                    NaN        NaN                NaN            NaN  
2                    NaN        NaN                NaN            NaN  
3                    NaN        NaN                NaN            NaN  
4                    NaN        NaN                NaN            NaN  
..                   ...        ...                ...        

In [ ]:
df_731b.[post_ob_process()

                                                           Part E             \
    Vis-à-vis country Vis-à-vis country name Derivative contracts Guarantees   
0                  AF            Afghanistan                  NaN        NaN   
1                  AL                Albania                  NaN        NaN   
2                  DZ                Algeria                  NaN        NaN   
3                  AS         American Samoa                  NaN        NaN   
4                  AD                Andorra                  NaN        NaN   
..                ...                    ...                  ...        ...   
235                WF      Wallis and Futuna                  NaN        NaN   
236                EH         Western Sahara                  NaN        NaN   
237                YE                  Yemen                  NaN        NaN   
238                ZM                 Zambia                  NaN        NaN   
239                ZW               Zimb

Traceback (most recent call last):
  File "c:\Users\Bing\anaconda3\Lib\site-packages\pandas\core\indexes\base.py", line 3791, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 152, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 181, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Vis-à-vis country'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Bing\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Bing\AppData\Local\Temp\ipykernel_16648\2174992493.py", line 1, in <module>
    df_731b.

In [ ]:
multi_index = [('' if col not in a.columns[3:] else 'Part A', col) for col in a.columns]
a.columns = pd.MultiIndex.from_tuples(multi_index)

In [ ]:
def add_top_header(df, header, start=2):
    multi_index = [('' if col not in df.columns[start:] else header , col) for col in df.columns]
    df.columns = pd.MultiIndex.from_tuples(multi_index)

In [ ]:
add_top_header(a, "Part A", 3)
add_top_header(b, "Part B")

In [ ]:
pd.concat([a, b], axis=1)

Part A               \
    Vis-à-vis country Vis-à-vis country name GL adj <=3 mths 3m-12m 1y-2y   
0                  AF            Afghanistan    NaN      NaN    NaN   NaN   
1                  AL                Albania    NaN      NaN    NaN   NaN   
2                  DZ                Algeria    NaN      NaN    NaN   NaN   
3                  AS         American Samoa    NaN      NaN    NaN   NaN   
4                  AD                Andorra    NaN      NaN    NaN   NaN   
..                ...                    ...    ...      ...    ...   ...   
235                WF      Wallis and Futuna    NaN      NaN    NaN   NaN   
236                EH         Western Sahara    NaN      NaN    NaN   NaN   
237                YE                  Yemen    NaN      NaN    NaN   NaN   
238                ZM                 Zambia    NaN      NaN    NaN   NaN   
239                ZW               Zimbabwe    NaN      NaN    NaN   NaN   

                                                ...         \
    2y-5y 5y+ Maturity bucket Adjustment Total  ... GL adj   
0     NaN NaN                        NaN   NaN  ...    NaN   
1     NaN NaN                        NaN   NaN  ...    NaN   
2     NaN NaN                        NaN   NaN  ...    NaN   
3     NaN NaN                        NaN   NaN  ...    NaN   
4     NaN NaN                        NaN   NaN  ...    NaN   
..    ...  ..                        ...   ...  ...    ...   
235   NaN NaN                        NaN   NaN  ...    NaN   
236   NaN NaN                        NaN   NaN  ...    NaN   
237   NaN NaN                        NaN   NaN  ...    NaN   
238   NaN NaN                        NaN   NaN  ...    NaN   
239   NaN NaN                        NaN   NaN  ...    NaN   

                  Part B                                   \
    Banking institutions Central banks General government   
0                    NaN           NaN                NaN   
1                    NaN           NaN                NaN   
2                    NaN           NaN                NaN   
3                    NaN           NaN                NaN   
4                    NaN           NaN                NaN   
..                   ...           ...                ...   
235                  NaN           NaN                NaN   
236                  NaN           NaN                NaN   
237                  NaN           NaN                NaN   
238                  NaN           NaN                NaN   
239                  NaN           NaN                NaN   

                                                                   \
    Non-banking financial institutions Non-financial corporations   
0                                  NaN                        NaN   
1                                  NaN                        NaN   
2                                  NaN                        NaN   
3                                  NaN                        NaN   
4                                  NaN                        NaN   
..                                 ...                        ...   
235                                NaN                        NaN   
236                                NaN                        NaN   
237                                NaN                        NaN   
238                                NaN                        NaN   
239                                NaN                        NaN   

                                                                             
    Households and non-profits Unallocated Total international claims Check  
0                          NaN         NaN                        NaN   NaN  
1                          NaN         NaN                        NaN   NaN  
2                          NaN         NaN                        NaN   NaN  
3                          NaN         NaN                        NaN   NaN  
4                          NaN         NaN                        NaN   NaN  
..     

In [ ]:
df_731b_e = df_731b_agg.get_part_e(df_facility_output)
aggregator.post_ob_process(df_ob_check, df_731b_e)

,Vis-à-vis country name,GL adj,Derivative contracts,Guarantees,Credit commitments,Guarantees adj
0,Afghanistan,NaN,NaN,NaN,NaN,NaN
1,Albania,NaN,NaN,NaN,NaN,NaN
2,Algeria,NaN,NaN,NaN,NaN,NaN
3,American Samoa,NaN,NaN,NaN,NaN,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
235,Wallis and Futuna,NaN,NaN,NaN,NaN,NaN
236,Western Sahara,NaN,NaN,NaN,NaN,NaN
237,Yemen,NaN,NaN,NaN,NaN,NaN
238,Zambia,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_ob_check

G                                          \
                    CCY Sum_AUD_Equivalent      Facility   diff   
0                   AUD      -1.606446e+09  1.606446e+09  0.000   
1                   CAD      -7.626023e+05  7.626023e+05  0.006   
2                   CNY      -3.025782e+08  3.025782e+08  1.402   
3                   EUR      -7.605264e+06  7.605264e+06 -0.012   
4                   HKD      -4.239931e+04  4.239931e+04  0.001   
5                   JPY       0.000000e+00           NaN    NaN   
6                   NZD       0.000000e+00           NaN    NaN   
7                   SGD      -2.421265e+06  2.421265e+06 -0.008   
8                   USD      -2.010825e+09  2.010825e+09 -5.569   
9           Grand Total      -3.930680e+09           NaN    NaN   
10               731.3B       3.930680e+09           NaN    NaN   
11  Guarantees Checking      -4.200000e+00           NaN    NaN   

                              F                                              
                            CCY Sum_AUD_Equivalent      Facility       diff  
0                           AUD      -5.234355e+09  5.234355e+09   0.000000  
1                           CAD       0.000000e+00  0.000000e+00   0.000000  
2                           CNY       0.000000e+00  0.000000e+00   0.000000  
3                           EUR      -7.156994e+07  7.156994e+07  -0.144501  
4                           GBP      -5.498837e+07  5.498837e+07  -0.011024  
5                           HKD       0.000000e+00  0.000000e+00   0.000000  
6                           NZD      -1.429209e+08  1.429209e+08   0.157148  
7                           SGD      -8.533358e+06  8.533358e+06  -0.033479  
8                           USD      -3.648583e+09  3.648583e+09 -10.119022  
9                   Grand Total      -9.160950e+09           NaN        NaN  
10                       731.3B       9.160950e+09           NaN        NaN  
11  Credit commitments Checking      -1.020000e+01           NaN        NaN

In [ ]:
df_731b_e

In [ ]:
df_731b_agg.get_part_b()

100%|██████████| 7/7 [00:00<00:00, 32.80it/s]


,Vis-à-vis country,Vis-à-vis country name,GL adj,Banking institutions,Central banks,General government,Non-banking financial institutions,Non-financial corporations,Households and non-profits,Unallocated,Total international claims,Check
0,AF,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DZ,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AS,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
235,WF,Wallis and Futuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,EH,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,YE,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,ZM,Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cols = ['<=3 mths', '3m-12m', '1y-2y', '2y-5y', '5y+', 'Maturity bucket Adjustment',
       'Total', 'Unallocated']
upper_level = ['Part a' for i in ]
df.columns = pd.MultiIndex.from_tuples([(upper, col) if upper else (col, '') for upper, col in zip(upper_level, df.columns)])

Index(['Vis-à-vis country', 'Vis-à-vis country name', 'GL adj', '<=3 mths',
       '3m-12m', '1y-2y', '2y-5y', '5y+', 'Maturity bucket Adjustment',
       'Total', 'Unallocated'],
      dtype='object')

In [ ]:
df_731b = df_731b_d.post_ob_process(df_ob_check)

In [ ]:
df_731b.columns

Index(['Vis-à-vis country', 'Vis-à-vis country name', 'GL adj', '<=3 mths',
       '3m-12m', '1y-2y', '2y-5y', '5y+', 'Maturity bucket Adjustment',
       'Total', 'Unallocated', 'Banking institutions', 'Central banks',
       'General government', 'Non-banking financial institutions',
       'Non-financial corporations', 'Households and non-profits',
       'Total international claims', 'Check', 'Outward risk transfer',
       'Inward risk transfer', 'Net transfer of risk to the ultimate borrower',
       'Derivative contracts', 'Guarantees', 'Credit commitments',
       'Guarantees adj'],
      dtype='object')

In [ ]:
['Vis-à-vis country', 'Vis-à-vis country name', 'GL adj',, , 
       'Derivative contracts', 'Guarantees', 'Credit commitments',
       'Guarantees adj']

In [ ]:
{
    'part a': ['<=3 mths','3m-12m', '1y-2y', '2y-5y', '5y+', 
               'Maturity bucket Adjustment','Total', 'Unallocated'],
    'part b': ['Banking institutions', 'Central banks','General government', 
               'Non-banking financial institutions', 'Non-financial corporations', 
               'Households and non-profits', 'Total international claims', 'Check'],
    'part c': ['Outward risk transfer','Inward risk transfer', 
               'Net transfer of risk to the ultimate borrower']
    'part d': []
}

In [ ]:
df_731b2 = df_731b.post_ob_process(df_ob_check)

In [ ]:
pd.DataFrame([None] * len(table1), columns=[" "])

,
0,None
1,None
2,None


add blank column


In [ ]:
df_731b2

,Vis-à-vis country,Vis-à-vis country name,GL adj,<=3 mths,3m-12m,1y-2y,2y-5y,5y+,Maturity bucket Adjustment,Total,...,Outward risk transfer,Inward risk transfer,Net transfer of risk to the ultimate borrower,Derivative contracts,Guarantees,Credit commitments,Guarantees adj,,,
0,AF,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None
1,AL,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None
2,DZ,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None
3,AS,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None
4,AD,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,WF,Wallis and Futuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None
236,EH,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None
237,YE,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None
238,ZM,Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,None,None,None


In [ ]:
df_mis_agg = df_mis_output[df_mis_output['G'] == 'G']\
            .groupby(['CCY'])\
            .agg(Sum_AUD_Equivalent=('AUD Equivalent', 'sum'))

In [ ]:
df_mis_agg['Sum_AUD_Equivalent'].sum(numeric_only=True)

-3930679892.39

In [ ]:
df_rt30_output[['Risk_in', 'Nationality','Vis-à-vis country', 'Risk Transfer']].to_clipboard(index=False)

In [ ]:
df_rt30_output[df_rt30_output.customer_nr.duplicated(keep=False)]

,ide_internal_party_ref,ide_linkage_ref,ide_linkage_type,ide_sourcesys_ref,lot_type_fk,rca_accrint,rca_bookv,rca_marketv,rca_prov_coll,rca_prov_indi,...,Key,fda_table,deal_type_derived,利息,估值,Nationality,rv_mat_remaining_derived,Risk_in,Risk Transfer,Risk in Counterparty
0,106,230808032270,12,FASM,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Non-financial corporations|AU,FEE,8475,NA,NA,AU,1060002,AU,N,Non-financial corporations
1,106,230808032277,12,FASM,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Non-financial corporations|AU,FEE,8475,NA,NA,AU,1060002,AU,N,Non-financial corporations
2,106,240523029514,12,FASM,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Non-financial corporations|AU,FEE,8475,NA,NA,AU,1060364,AU,N,Non-financial corporations
3,106,221229018953,12,FASM,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Non-financial corporations|AU,FEE,8475,NA,NA,AU,1060353,AU,N,Non-financial corporations
4,106,190725019311,12,FASM,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Non-financial corporations|AU,FEE,8475,NA,NA,AU,1059336,AU,N,Non-financial corporations
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24907,106,328752667,8,22,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Banking: related offices|HK,FOREX,0141,NA,8832,CN,1003000,CN,Y,Banking: related offices
24908,106,329214259,8,22,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Banking: related offices|HK,FOREX,0141,NA,7832,CN,1003000,CN,Y,Banking: related offices
24909,106,329212211,8,22,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Banking: related offices|HK,FOREX,0141,NA,7832,CN,1003000,CN,Y,Banking: related offices
24910,106,328969679,8,22,992,0.00,0.0,0.0,0.0,0.0,...,AUD|Banking: related offices|HK,FOREX,0141,NA,8832,CN,1003000,CN,Y,Banking: related offices


In [ ]:
# Create your sample tables
table1 = pd.DataFrame({
    "A": [1, 2, 3],
    "B": [4, 5, 6],
    "C": [7, 8, 9]
})

table2 = pd.DataFrame({
    "D": ["a", "b", "c"],
    "E": ["d", "e", "f"],
    "F": ["g", "h", "i"]
})

# Add a blank column to separate tables
blank_column = pd.DataFrame([None] * len(table1), columns=[" "])

# Combine tables into a single DataFrame
combined = pd.concat([table1, blank_column, table2], axis=1)

In [ ]:
combined.to_excel('test.xlsx', index=False)

In [ ]:
df_731b[
    df_731b['Vis-à-vis country'].notna() & df_731b['Vis-à-vis country'] != None
]

,Vis-à-vis country,Vis-à-vis country name,GL adj,<=3 mths,3m-12m,1y-2y,2y-5y,5y+,Manual Adjustment,Maturity bucket Adjustment,Total,Unallocated
0,AF,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
1,AL,Albania,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,DZ,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,AS,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
4,AD,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
235,WF,Wallis and Futuna,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
236,EH,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
237,YE,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
238,ZM,Zambia,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [ ]:
rt30_agg2 = (df_rt30_output[
    (df_rt30_output.ALE == 'A') &
    (df_rt30_output.rv_mat_remaining > 1060000)]
        .groupby('Vis-à-vis country')['rca_deferred_fee']
        .sum()
        .reset_index()
)

In [ ]:
def add_maturity_bucket_adjustment(self):
        
        rt30_filtered = - self.rt30[
            (self.rt30['ALE'] == 'A') &
            (self.rt30['rv_mat_remaining'] > 1060000)
            ]
        
        rt30_agg = rt30_filtered\
            .groupby('Vis-à-vis country')\
            .agg(maturity_bucket_adjustment=('rca_bookv', 'sum'))
        
        self.output = self.output.merge(rt30_agg, on='Vis-à-vis country', how='left')
        
        return self

,Vis-à-vis country,rca_deferred_fee
0,AU,0.0
1,BM,0.0
2,CA,0.0
3,CN,0.0
4,DE,0.0
5,GB,0.0
6,HK,0.0
7,JE,0.0
8,KY,0.0
9,LU,0.0


In [ ]:
4360084491 -41711

4360042780

In [ ]:
base_mask = df_rt30_output['ALE'] == 'A'
condition = (
    (df_rt30_output['rv_mat_remaining_derived'] > 1000000) & 
    (df_rt30_output['rv_mat_remaining_derived'] <= 1003000)
)
# base_mask &= condition

rt30_filtered = df_rt30_output[base_mask]

In [ ]:
base_mask

0        False
1        False
2        False
3        False
4        False
         ...  
24907    False
24908    False
24909    False
24910    False
24911    False
Name: ALE, Length: 24912, dtype: bool

In [ ]:
rt30_agg = (rt30_filtered
                    .groupby('Vis-à-vis country')
                    .agg({
                        'rca_bookv': 'sum',
                        'rca_accrint': 'sum',
                        'rca_deferred_fee': 'sum',
                    })
                    .reset_index()
)

sum_8754 = (
            rt30_filtered[rt30_filtered['deal_type_derived'] == '8754']
            .groupby('Vis-à-vis country')['rca_bookv']
            .sum()
            .reset_index()
            .rename(columns={'rca_bookv': '8754_bookv'})
)

In [ ]:
result = rt30_agg.merge(sum_8754, on='Vis-à-vis country', how='left').fillna({'8754_bookv':0})

In [ ]:
result['output'] = (
            result['rca_bookv'] - 
            2 * result['8754_bookv'] - 
            result['rca_deferred_fee'] +
            result['rca_accrint']
        )

In [ ]:
from validation import DervsGL

derivs = DervsGL(mapper, df_rt30_outout, df_mis_output)
rca_mtm = derivs.process_all()


2025-01-16 19:35:26,169 - mapping - INFO - Applying mapping using keys: 核算码 -> AP Code
2025-01-16 19:35:26,173 - mapping - INFO - Duplicate entries found in mapping for '['AP Code']':
4
2025-01-16 19:35:26,178 - mapping - INFO - Dropped duplicates from mapping for '['AP Code']'


In [ ]:
rca_mtm['1'].loc[len(rca_mtm['1'])] = ' '

In [ ]:
rca_mtm['1']

,核算码,核算码名称,rca_mtm,DM010,Diff
0,7444,待入衍生费用,NaN,-81958.7,-81958.7
1,7801,买入外汇期权公允价值变动,12179.8,69778.9,81958.7
2,7802,境外分行买入外汇期权公允价值变动,NaN,0.0,0.0
3,TOTAL,NaN,NaN,NaN,0.0
4,,,,,


In [ ]:
for k, v in rca_mtm.items():
    v.loc[len(v)] = ''
    

In [ ]:
output = None 

for key, group in rca_mtm.groupby('group'):
    total_row = pd.DataFrame([{'group': 'TOTAL','Diff': group['Diff'].sum()}])
    blank_row = pd.DataFrame([{'group':None}])
    output = pd.concat([output, group, total_row, blank_row])
    

In [ ]:
output.to_excel('test.xlsx', index=False)

In [ ]:
pd.concat([group, total_row, pd.DataFrame([{'group':'1'}])])

,group,核算码,核算码名称,rca_mtm,DM010,Diff
0,1,7444,待入衍生费用,NaN,-81958.7,-81958.7
1,1,7801,买入外汇期权公允价值变动,12179.8,69778.9,81958.7
2,1,7802,境外分行买入外汇期权公允价值变动,NaN,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN
4,,,,,,
5,,,,,,
0,TOTAL,NaN,NaN,NaN,NaN,0.0
0,1,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.concat([group, total_row, pd.DataFrame([{'核算码':None}])])

,group,核算码,核算码名称,rca_mtm,DM010,Diff
0,1,7444,待入衍生费用,NaN,-81958.7,-81958.7
1,1,7801,买入外汇期权公允价值变动,12179.8,69778.9,81958.7
2,1,7802,境外分行买入外汇期权公允价值变动,NaN,0.0,0.0
0,TOTAL,NaN,NaN,NaN,NaN,0.0
0,NaN,None,NaN,NaN,NaN,NaN


In [ ]:
total_row = pd.DataFrame([{'group': 'TOTAL','Diff': group['Diff'].sum()}])

pd.concat([group, total_row])

,group,核算码,核算码名称,rca_mtm,DM010,Diff
0,1,7444,待入衍生费用,NaN,-81958.7,-81958.7
1,1,7801,买入外汇期权公允价值变动,12179.8,69778.9,81958.7
2,1,7802,境外分行买入外汇期权公允价值变动,NaN,0.0,0.0
0,TOTAL,NaN,NaN,NaN,NaN,0.0


# Database connection


In [ ]:
from sqlalchemy import create_engine
from queries import RT30_RT32_SQL

with create_engine('mssql+pyodbc://DESKTOP-Q79EBHP/BOC?driver=ODBC+Driver+17+for+SQL+Server') as engine:
    
    df_rt30_rt32 = pd.read_sql(RT30_RT32_SQL, engine)
